In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
project_path = '/kaggle/input/new-york-city-taxi-fare-prediction/'
train = pd.read_csv(project_path + 'train.csv',nrows=5000000)
test = pd.read_csv(project_path + 'test.csv')
test_ids = test['key']
train.head()


In [ ]:
train.info()


In [ ]:
train.describe()


In [ ]:
train.fare_amount.hist(bins=100,figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")


In [ ]:
train[train.fare_amount <100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")


In [ ]:
train[train.fare_amount >=100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")


In [ ]:
train[train.fare_amount <100].shape


In [ ]:
train[train.fare_amount >=100].shape


In [ ]:
train.passenger_count.hist(bins=100,figsize = (16,8))
plt.xlabel("passenger_count")
plt.ylabel("Frequency")


In [ ]:
train[train.passenger_count<10].passenger_count.hist(bins=10,figsize = (16,8))
plt.xlabel("passenger_count")
plt.ylabel("Frequency")


In [ ]:
train[train.passenger_count<7].passenger_count.hist(bins=10,figsize = (16,8))
plt.xlabel("passenger_count")
plt.ylabel("Frequency")


In [ ]:
train[train.passenger_count>7].passenger_count.hist(bins=10,figsize = (16,8))
plt.xlabel("passenger_count")
plt.ylabel("Frequency")


In [ ]:
train[train.passenger_count >7]


In [ ]:
train[train.passenger_count ==0].shape


In [ ]:
plt.figure(figsize= (16,8))
sns.boxplot(x = train[train.passenger_count< 7].passenger_count, y = train.fare_amount)


In [ ]:
train[train.passenger_count <7][['fare_amount','passenger_count']].corr()


In [ ]:
train.isnull().sum() # find null value


In [ ]:
train = train.dropna(how='any', axis=0) # delete them


In [ ]:
test.head()
test.isnull().sum()


In [ ]:
train = train[train.fare_amount>=0]


In [ ]:
def deal_time_features(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df['hour'] = df.pickup_datetime.dt.hour
    df['month'] = df.pickup_datetime.dt.month
    df["year"] = df.pickup_datetime.dt.year
    df["weekday"] = df.pickup_datetime.dt.weekday
    return df
train = deal_time_features(train)
test = deal_time_features(test)

train.head()


In [ ]:
def distance(x1, y1, x2, y2):
    p = 0.017453292519943295 
    a = 0.5 - np.cos((x2 - x1) * p)/2 + np.cos(x1 * p) * np.cos(x2 * p) * (1 - np.cos((y2 - y1) * p)) / 2
    dis = 0.6213712 * 12742 * np.arcsin(np.sqrt(a))
    return dis  
train['distance_miles'] = distance(train.pickup_latitude,train.pickup_longitude,train.dropoff_latitude,train.dropoff_longitude)
test['distance_miles'] = distance(test.pickup_latitude, test.pickup_longitude,test.dropoff_latitude,test.dropoff_longitude)
train.head()


In [ ]:
train[(train['distance_miles']==0)&(train['fare_amount']==0)] # Print data with price and distance of 0
train = train.drop(index= train[(train['distance_miles']==0)&(train['fare_amount']==0)].index, axis=0) # delete them


In [ ]:
train.describe().T


In [ ]:
train.drop(columns=['key', 'pickup_datetime'], axis=1).corr()['fare_amount']


In [ ]:
df_train = train.drop(columns=['key', 'pickup_datetime'], axis=1).copy()
df_test = test.drop(columns=['key', 'pickup_datetime'], axis=1).copy()

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('fare_amount',axis=1)
                                                    ,df_train['fare_amount']
                                                    ,test_size=0.2
                                                    ,random_state = 42)

import xgboost as xgb
params = { 
    'max_depth': 7,
    'gamma' :0,
    'eta':0.3, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
}
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=5000, 
                    early_stopping_rounds=10,
                    evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)


In [ ]:
prediction = model.predict(xgb.DMatrix(df_test))
prediction


In [ ]:
res = pd.DataFrame()
res['key'] = test['key']
res['fare_amount'] = prediction
res.to_csv('/kaggle/working/results.csv', index=False)
